In [2]:
import numpy as np
np.random.seed(5) 
import tensorflow as tf
#tf.set_random_seed(2)
import matplotlib.pyplot as plt
%matplotlib inline
import os
import cv2
import keras
from sklearn.model_selection import train_test_split

train_dir = '../input/asl-alphabet/asl_alphabet_train/asl_alphabet_train'
eval_dir = '../input/asl-alphabet/asl_alphabet_test/asl_alphabet_test'


#2. Loading the data
#Helper function to load images from given directories
def load_images(directory,uniq_labels):
    images = []
    labels = []
    for idx, label in enumerate(uniq_labels):
        if (directory == train_dir):
            for file in os.listdir(directory + "/" + label):
                filepath = directory + "/" + label + "/" + file
                #image = cv2.resize(cv2.imread(filepath), (64, 64))
                image = cv2.imdecode(np.fromfile(filepath, dtype=np.uint8), cv2.IMREAD_UNCHANGED)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                image = cv2.resize(image, (64, 64))
                images.append(image)
                labels.append(idx)
        else:
            filepath = directory + "/" + label 
            #image = cv2.resize(cv2.imread(filepath), (64, 64))
            image = cv2.imdecode(np.fromfile(filepath, dtype=np.uint8), cv2.IMREAD_UNCHANGED)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            image = cv2.resize(image, (64, 64))
            images.append(image)
            labels.append(idx)
    images = np.array(images)
    labels = np.array(labels)
    return(images, labels)

Using TensorFlow backend.


In [ ]:
CATEGORIES = sorted(os.listdir(train_dir))
#read images in train folder
images, labels = load_images(directory = train_dir, uniq_labels = CATEGORIES)
#read images in test folder
X_eval, y_eval = load_images(directory = eval_dir, uniq_labels = sorted(os.listdir(eval_dir)))


In [ ]:
#split data to train and test
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size = 0.1, stratify = labels)

n = len(sorted(os.listdir(train_dir)))
train_n = len(X_train)
test_n = len(X_test)
eval_n = len(X_eval)

print("Total number of symbols: ", n)
print("Number of training images: " , train_n)
print("Number of testing images: ", test_n)
print("Number of evaluation images: ", eval_n)

In [ ]:
X_train.shape

In [ ]:
#preprocessing
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)
y_eval = keras.utils.to_categorical(y_eval)

X_train = X_train.astype('float32')/255.0
X_test = X_test.astype('float32')/255.0
X_eval = X_eval.astype('float32')/255.0
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)

In [ ]:
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Conv2D, Dense, Dropout, Flatten
from keras.layers import Flatten, Dense
from keras.models import Sequential
#build the model
model = Sequential()
model.add(Conv2D(filters = 64, kernel_size = 5, padding = 'same', activation = 'relu', input_shape = (64, 64, 1)))
model.add(Conv2D(filters = 64, kernel_size = 5, padding = 'same', activation = 'relu'))
model.add(MaxPooling2D(pool_size = (4, 4)))
model.add(Dropout(0.5))
model.add(Conv2D(filters = 128 , kernel_size = 5, padding = 'same', activation = 'relu'))
model.add(Conv2D(filters = 128 , kernel_size = 5, padding = 'same', activation = 'relu'))
model.add(MaxPooling2D(pool_size = (4, 4)))
model.add(Dropout(0.5))
model.add(Conv2D(filters = 256 , kernel_size = 5, padding = 'same', activation = 'relu'))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(29, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
#compil the model
model.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy', metrics = ['accuracy'])
#fit the model
hist = model.fit(X_train, y_train, epochs = 5, batch_size = 64)

In [ ]:
#save model
model.save('ASLGray.model')

#load model
model=tf.keras.models.load_model('ASLGray.model')

In [1]:
#Download model from kaggle
from IPython.display import FileLink
FileLink('ASLGray.model')

/kaggle/working/ASLGray.model

In [ ]:
#Accuracy of model
score = model.evaluate(x = X_test, y = y_test, verbose = 0)
print('Accuracy for test images:', round(score[1]*100, 3), '%')

In [ ]:
#prepare image to prediction
def prepare(filepath):
    image = cv2.imdecode(np.fromfile(filepath, dtype=np.uint8), cv2.IMREAD_UNCHANGED)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.resize(image, (64, 64))
    image=image.reshape(-1, 64, 64, 1)
    image=image.astype('float32')/255.0
    return  image

#use this function to predict images
def predict(my_model, filepath):
    prediction = model.predict([prepare(filepath)]) 
    category = np.argmax(prediction[0])
    return  CATEGORIES[category]

In [ ]:
category = predict(model,'/kaggle/input/asl-alphabet/asl_alphabet_train/asl_alphabet_train/W/W2348.jpg')
print("The image class is: " + str(category))

In [ ]:
category = predict(model,'/kaggle/input/asl-alphabet/asl_alphabet_train/asl_alphabet_train/L/L11.jpg')
print("The image class is: " + str(category))

In [ ]:
category = predict(model,'/kaggle/input/asl-alphabet/asl_alphabet_test/asl_alphabet_test/A_test.jpg')
print("The image class is: " + str(category))

In [ ]:
category = predict(model,'/kaggle/input/asl-alphabet/asl_alphabet_test/asl_alphabet_test/B_test.jpg')
print("The image class is: " + str(category))

In [ ]:
category = predict(model,'/kaggle/input/asl-alphabet/asl_alphabet_test/asl_alphabet_test/C_test.jpg')
print("The image class is: " + str(category))

In [ ]:
category = predict(model,'/kaggle/input/asl-alphabet/asl_alphabet_test/asl_alphabet_test/D_test.jpg')
print("The image class is: " + str(category))